In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [ ]:
!rm -r Machine-Learning/
!git clone https://github.com/Sintry1/Machine-learning

rm: cannot remove 'Machine-Learning/': No such file or directory
Cloning into 'Machine-learning'...
remote: Enumerating objects: 59401, done.
remote: Total 59401 (delta 0), reused 0 (delta 0), pack-reused 59401
Receiving objects: 100% (59401/59401), 1.31 GiB | 36.32 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Updating files: 100% (59377/59377), done.


In [ ]:
trainingSet = '/content/Machine-learning/seg_train/seg_train'
testSet = '/content/Machine-learning/seg_test/seg_test'

In [ ]:
testSet

'/content/Machine-learning/seg_test/seg_test'

In [ ]:
train_data = ImageDataGenerator(
        rescale=1./255, # Change pixel value from 0-255 to 0.0 - 1.0 This applies Feature Scaling to every image, by dividing the pixel values by 255
        shear_range=0.2, # Adding a shear_range so that images can be processed from different perspectives
        zoom_range=0.2, # How far to randomly zoom in/out of an image
        horizontal_flip=True) # This will randomly flip our input images horizontally.

training_set = train_data.flow_from_directory(
        trainingSet, # Path to folder with images
        target_size=(150, 150), # This is how big our images will be when they are fed into the CNN
        batch_size=32, # This is how many "rows" or images it will go through in each Epoch before backpropogating
        class_mode='categorical') # use 'categorical' for > 2 class, 'binary' for two-class problems

Found 14034 images belonging to 6 classes.


In [ ]:
test_data = ImageDataGenerator(rescale=1./255)

test_set = train_data.flow_from_directory(
        testSet, # Path to folder with images
        target_size=(150, 150), # This is how big our images will be when they are fed into the CNN
        batch_size=32, # This is how many "rows" or images it will go through in each Epoch before backpropogating
        class_mode='categorical') # use 'categorical' for > 2 class, 'binary' for two-class problems

Found 3000 images belonging to 6 classes.


In [ ]:
model = tf.keras.models.Sequential() # instantiate new Sequential object

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=6, activation="relu", input_shape=[150, 150, 3])) #Conv2D needs 3 parameters;
# 1: filters, which is the number of feature detectors.
# 2: kernel_size, which is the size of feature detectors
# 3: activation. Finally there's a 4th we should add, which is input_shape. input_shape itself takes 3 parameters. (height, width, dimensions)
# input_shape is only needed on the first layer we add, as it determines the size of the images and the dimenions (3 dimensions = color i.e. RGB, 1 dimension = black & white)
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=6, activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Here we have to add 2 parameters;
# 1: pool_size, which refers to the feature detector we use when we apply max pooling
# 2: strides, which refers to how large our stride should be. This should usually match the pool_size variable too.

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

model.add(tf.keras.layers.Flatten()) # No parameters needed here, as it is smart enough to know that flattening should be applied to the previous 3 layers.

model.add(tf.keras.layers.Dense(units=256, activation="relu")) # add a fully connected layer

model.add(tf.keras.layers.Dense(units=6, activation="sigmoid")) # We change the activation function to sigmoid here because it is the output layer.
# We only need 1 unit because the output is binary (cat or dog/0 or 1)

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy']) # This method from the Sequential class library compiles our ANN and needs 3 parameters.
# The first parameter is the optimiser, the second is the loss function, and the third is the metric(s) such as accuracy.
# adam as an optimizer allows us to perform stochastic gradient descent
# If you're doing binary classification (like this) the loss function must always be 'binary_crossentropy'
# If we were doing non-binary classification, it would need to be 'categorical_crossentropy'
# Since we can select multiple metrics, we have to enter them in [], however we're only looking at accuracy here.

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)

In [ ]:
model.fit(x=training_set, validation_data=test_set, epochs=25, callbacks=[earlyStop])

Epoch 1/25
439/439 [==============================] - 136s 273ms/step - loss: 1.1217 - accuracy: 0.5438 - val_loss: 0.9471 - val_accuracy: 0.6183
Epoch 2/25
439/439 [==============================] - 114s 259ms/step - loss: 0.7827 - accuracy: 0.7033 - val_loss: 0.6906 - val_accuracy: 0.7373
Epoch 3/25
439/439 [==============================] - 114s 258ms/step - loss: 0.6831 - accuracy: 0.7440 - val_loss: 0.7052 - val_accuracy: 0.7373
Epoch 4/25
439/439 [==============================] - 112s 255ms/step - loss: 0.6022 - accuracy: 0.7751 - val_loss: 0.6131 - val_accuracy: 0.7670
Epoch 5/25
439/439 [==============================] - 115s 262ms/step - loss: 0.5497 - accuracy: 0.8003 - val_loss: 0.5277 - val_accuracy: 0.8067
Epoch 6/25
439/439 [==============================] - 113s 258ms/step - loss: 0.5074 - accuracy: 0.8179 - val_loss: 0.5281 - val_accuracy: 0.8173
Epoch 7/25
439/439 [==============================] - 114s 260ms/step - loss: 0.4902 - accuracy: 0.8230 - val_loss: 0.6009 -